Notebook by:

* Lorenzo Pannacci 1948926
* ADD YOUR NAME

## Startup

In [34]:
######################
# LIBRARIES DOWNLOAD #
######################

install_packages = False
if install_packages:
    %pip install numpy pandas

In [35]:
####################
# LIBRARIES IMPORT #
####################

import numpy as np
import pandas as pd

In [36]:
#####################
# GLOBAL PARAMETERS #
#####################

dataset_path = r"data/vodclickstream_uk_movies_03.csv"

## 1. Recommendation sytem 

Implementing a recommendation system is critical for businesses and digital platforms that want to thrive in today's competitive environment. These systems use data-driven personalization to tailor content, products, and services to individual user preferences. The latter improves user engagement, satisfaction, retention, and revenue through increased sales and cross-selling opportunities. In this section, you will attempt to implement a recommendation system by identifying similar users' preferences and recommending movies they watch to the study user. 

To be more specific, you will implement your version of the [**LSH algorithm**](https://www.learndatasci.com/tutorials/building-recommendation-engine-locality-sensitive-hashing-lsh-python/), which will take as input the user's preferred genre of movies, find the most similar users to this user, and recommend the most watched movies by those who are more similar to the user. 

__Data__: The data you will be working with can be found [here](https://www.kaggle.com/datasets/vodclickstream/netflix-audience-behaviour-uk-movies).

Looking at the data, you can see that there is data available for each user for the movies the user <ins>clicked on</ins>. Gather the __title and genre__ of the __maximum top 10 movies__ that each user clicked on regarding the __number of clicks__.

In [37]:
# TODO

### 1.2 Minhash Signatures

Using the movie genre and user_ids, try to implement your min-hash signatures so that users with similar interests in a genre appear in the same bucket. 

__Important note:__ You must write your minhash function from scratch.  You are not permitted to use any already implemented hash functions.  Read the class materials and, if necessary, conduct an internet search.  The description of hash functions in the [book](http://infolab.stanford.edu/~ullman/mmds/ch3n.pdf) may be helpful as a reference.

In [38]:
# TODO

### 1.3 Locality-Sensitive Hashing (LSH)

Now that your buckets are ready, it's time to ask a few queries. We will provide you with some user_ids and ask you to recommend at __most five movies__ to the user to watch based on the movies clicked by similar users. 

To recommend at most five movies given a user_id, use the following procedure: 

1. Identify the <ins>two most similar</ins> users to this user.
2. If these two users have any movies __in common__, recommend those movies based on the total number of clicks by these users.
3. If there are __no more common__ movies, try to propose the most clicked movies by the __most similar user first__, followed by the other user. 

__Note:__ At the end of the process, we expect to see at most five movies recommended to the user.

__Example:__ assume you've identified user __A__ and __B__ as the most similar users to a single user, and we have the following records on these users: 

- User A with 80% similarity
- User B with 50% similarity
  
|user|movie title|#clicks|
|---|---|---|
|A|Wild Child|20|
|A|Innocence|10|
|A|Coin Heist|2|
|B|Innocence|30|
|B|Coin Heist|15|
|B|Before I Fall|30|
|B|Beyond Skyline|8|
|B|The Amazing Spider-Man|5|

- __Recommended movies__ in order:
    - Innocence
    - Coin Heist
    - Wild Child
    - Before I Fall
    - Beyond Skyline

In [39]:
# TODO

## 2. Grouping Users together!

Now, we will deal with clustering algorithms that will provide groups of Netflix users that are similar among them.

To solve this task, you must accomplish the following stages:

### 2.1 Getting your data + feature engineering

1)  Access to the data found in [this dataset](https://www.kaggle.com/datasets/vodclickstream/netflix-audience-behaviour-uk-movies)

2)  Sometimes, the features (variables, fields) are not given in a dataset but can be created from it; this is known as *feature engineering*. For example, the original dataset has several clicks done by the same user, so grouping data by user_id will allow you to create new features **for each** user:

    a)  Favorite genre (i.e., the genre on which the user spent the most time)

    b)  Average click duration

    c)  Time of the day (Morning/Afternoon/Night) when the user spends the most time on the platform (the time spent is tracked through the duration of the clicks)

    d)  Is the user an old movie lover, or is he into more recent stuff (content released after 2010)?

    e)  Average time spent a day by the user (considering only the days he logs in)

So, in the end, you should have for each user_id five features.

3)  Consider at least 10 additional features that can be generated for each user_id (you can use chatGPT or other LLM tools for suggesting features to create). Describe each of them and add them to the previous dataset you made (the one with five features). In the end, you should have for each user at least 15 features (5 recommended + 10 suggested by you).

---

We start by exploring our data and convert it in a more usable form. The dataset is light, we don't have to worry about dividing it into chunks.

---

In [40]:
# load the dataset from the .json file to a pandas dataframe
original_dataset = pd.read_csv(dataset_path)
original_dataset

,Unnamed: 0,datetime,duration,title,genres,release_date,movie_id,user_id
0,58773,2017-01-01 01:15:09,0.0,"Angus, Thongs and Perfect Snogging","Comedy, Drama, Romance",2008-07-25,26bd5987e8,1dea19f6fe
1,58774,2017-01-01 13:56:02,0.0,The Curse of Sleeping Beauty,"Fantasy, Horror, Mystery, Thriller",2016-06-02,f26ed2675e,544dcbc510
2,58775,2017-01-01 15:17:47,10530.0,London Has Fallen,"Action, Thriller",2016-03-04,f77e500e7a,7cbcc791bf
3,58776,2017-01-01 16:04:13,49.0,Vendetta,"Action, Drama",2015-06-12,c74aec7673,ebf43c36b6
4,58777,2017-01-01 19:16:37,0.0,The SpongeBob SquarePants Movie,"Animation, Action, Adventure, Comedy, Family, ...",2004-11-19,a80d6fc2aa,a57c992287
...,...,...,...,...,...,...,...,...
671731,730504,2019-06-30 21:37:08,851.0,Oprah Presents When They See Us Now,Talk-Show,2019-06-12,43cd23f30f,57501964fd
671732,730505,2019-06-30 21:49:34,91157.0,HALO Legends,"Animation, Action, Adventure, Family, Sci-Fi",2010-02-16,febf42d55f,d4fcb079ba
671733,730506,2019-06-30 22:00:44,0.0,Pacific Rim,"Action, Adventure, Sci-Fi",2013-07-12,7b15e5ada1,4a14a2cd5a
671734,730507,2019-06-30 22:04:23,0.0,ReMastered: The Two Killings of Sam Cooke,"Documentary, Music",2019-02-08,52d49c515a,0b8163ea4b


In [41]:
original_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 671736 entries, 0 to 671735
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Unnamed: 0    671736 non-null  int64  
 1   datetime      671736 non-null  object 
 2   duration      671736 non-null  float64
 3   title         671736 non-null  object 
 4   genres        671736 non-null  object 
 5   release_date  671736 non-null  object 
 6   movie_id      671736 non-null  object 
 7   user_id       671736 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 41.0+ MB


---

The first thing we notice is the presence of a column called `Unnamed: 0` that has an incremental value. We can discard it and uses our index that starts from 0:

---

In [42]:
original_dataset.drop('Unnamed: 0', axis = 1, inplace = True)
original_dataset

,datetime,duration,title,genres,release_date,movie_id,user_id
0,2017-01-01 01:15:09,0.0,"Angus, Thongs and Perfect Snogging","Comedy, Drama, Romance",2008-07-25,26bd5987e8,1dea19f6fe
1,2017-01-01 13:56:02,0.0,The Curse of Sleeping Beauty,"Fantasy, Horror, Mystery, Thriller",2016-06-02,f26ed2675e,544dcbc510
2,2017-01-01 15:17:47,10530.0,London Has Fallen,"Action, Thriller",2016-03-04,f77e500e7a,7cbcc791bf
3,2017-01-01 16:04:13,49.0,Vendetta,"Action, Drama",2015-06-12,c74aec7673,ebf43c36b6
4,2017-01-01 19:16:37,0.0,The SpongeBob SquarePants Movie,"Animation, Action, Adventure, Comedy, Family, ...",2004-11-19,a80d6fc2aa,a57c992287
...,...,...,...,...,...,...,...
671731,2019-06-30 21:37:08,851.0,Oprah Presents When They See Us Now,Talk-Show,2019-06-12,43cd23f30f,57501964fd
671732,2019-06-30 21:49:34,91157.0,HALO Legends,"Animation, Action, Adventure, Family, Sci-Fi",2010-02-16,febf42d55f,d4fcb079ba
671733,2019-06-30 22:00:44,0.0,Pacific Rim,"Action, Adventure, Sci-Fi",2013-07-12,7b15e5ada1,4a14a2cd5a
671734,2019-06-30 22:04:23,0.0,ReMastered: The Two Killings of Sam Cooke,"Documentary, Music",2019-02-08,52d49c515a,0b8163ea4b


---

It will be useful to convert both the the `datetime` and the `release_date` columns from a string to actual datetime format. We notice that some entries of `release_date` have value `NOT AVAILABLE`, we will have to convert them to NaT (Not a Time).

---

In [43]:
original_dataset.datetime = pd.to_datetime(original_dataset.datetime, errors='coerce')
original_dataset.release_date = pd.to_datetime(original_dataset.release_date, errors='coerce')
original_dataset

,datetime,duration,title,genres,release_date,movie_id,user_id
0,2017-01-01 01:15:09,0.0,"Angus, Thongs and Perfect Snogging","Comedy, Drama, Romance",2008-07-25,26bd5987e8,1dea19f6fe
1,2017-01-01 13:56:02,0.0,The Curse of Sleeping Beauty,"Fantasy, Horror, Mystery, Thriller",2016-06-02,f26ed2675e,544dcbc510
2,2017-01-01 15:17:47,10530.0,London Has Fallen,"Action, Thriller",2016-03-04,f77e500e7a,7cbcc791bf
3,2017-01-01 16:04:13,49.0,Vendetta,"Action, Drama",2015-06-12,c74aec7673,ebf43c36b6
4,2017-01-01 19:16:37,0.0,The SpongeBob SquarePants Movie,"Animation, Action, Adventure, Comedy, Family, ...",2004-11-19,a80d6fc2aa,a57c992287
...,...,...,...,...,...,...,...
671731,2019-06-30 21:37:08,851.0,Oprah Presents When They See Us Now,Talk-Show,2019-06-12,43cd23f30f,57501964fd
671732,2019-06-30 21:49:34,91157.0,HALO Legends,"Animation, Action, Adventure, Family, Sci-Fi",2010-02-16,febf42d55f,d4fcb079ba
671733,2019-06-30 22:00:44,0.0,Pacific Rim,"Action, Adventure, Sci-Fi",2013-07-12,7b15e5ada1,4a14a2cd5a
671734,2019-06-30 22:04:23,0.0,ReMastered: The Two Killings of Sam Cooke,"Documentary, Music",2019-02-08,52d49c515a,0b8163ea4b


In [44]:
original_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 671736 entries, 0 to 671735
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   datetime      671736 non-null  datetime64[ns]
 1   duration      671736 non-null  float64       
 2   title         671736 non-null  object        
 3   genres        671736 non-null  object        
 4   release_date  641432 non-null  datetime64[ns]
 5   movie_id      671736 non-null  object        
 6   user_id       671736 non-null  object        
dtypes: datetime64[ns](2), float64(1), object(4)
memory usage: 35.9+ MB


---

The Kaggle page of the dataset (https://www.kaggle.com/datasets/vodclickstream/netflix-audience-behaviour-uk-movies) give us informations about the `duration` column:

`'duration' shows how long it was (in seconds) until that user clicked on another URL. A watch time of zero seconds means they visited the page but instantly clicked away.`

It is already a float, we don't need to treat it any further.

We can check the consistency of the dataset by checking if the same movie (i.e. all the entries with same `movie_id`) have the same characteristics (i.e. same `title`, `genres` and `release date`):

---

In [45]:
# iterate trough every movie
for movie_id, group_data in original_dataset.groupby('movie_id'):
    
    # check if there is more than a title for the same movie
    if group_data.title.unique().shape[0] != 1:
        print("Movie", movie_id, "has more than a title!")

    # check if there is more than a genre list for the same movie
    if group_data.genres.unique().shape[0] != 1:
        print("Movie", movie_id, "has conflicting genres!")

    # check if there is more than a release date for the same movie
    if group_data.release_date.unique().shape[0] != 1:
        print("Movie", movie_id, "has conflicting release dates!")

---

Since the `genres` column is a string we have to convert it to a list.

---

In [46]:
original_dataset.genres = original_dataset.genres.apply(lambda row: row.split(','))
original_dataset

,datetime,duration,title,genres,release_date,movie_id,user_id
0,2017-01-01 01:15:09,0.0,"Angus, Thongs and Perfect Snogging","[Comedy, Drama, Romance]",2008-07-25,26bd5987e8,1dea19f6fe
1,2017-01-01 13:56:02,0.0,The Curse of Sleeping Beauty,"[Fantasy, Horror, Mystery, Thriller]",2016-06-02,f26ed2675e,544dcbc510
2,2017-01-01 15:17:47,10530.0,London Has Fallen,"[Action, Thriller]",2016-03-04,f77e500e7a,7cbcc791bf
3,2017-01-01 16:04:13,49.0,Vendetta,"[Action, Drama]",2015-06-12,c74aec7673,ebf43c36b6
4,2017-01-01 19:16:37,0.0,The SpongeBob SquarePants Movie,"[Animation, Action, Adventure, Comedy, Fam...",2004-11-19,a80d6fc2aa,a57c992287
...,...,...,...,...,...,...,...
671731,2019-06-30 21:37:08,851.0,Oprah Presents When They See Us Now,[Talk-Show],2019-06-12,43cd23f30f,57501964fd
671732,2019-06-30 21:49:34,91157.0,HALO Legends,"[Animation, Action, Adventure, Family, Sci...",2010-02-16,febf42d55f,d4fcb079ba
671733,2019-06-30 22:00:44,0.0,Pacific Rim,"[Action, Adventure, Sci-Fi]",2013-07-12,7b15e5ada1,4a14a2cd5a
671734,2019-06-30 22:04:23,0.0,ReMastered: The Two Killings of Sam Cooke,"[Documentary, Music]",2019-02-08,52d49c515a,0b8163ea4b


---

We want to build a new dataset of user features extracted from the current one. We start by creating a dataframe containing only `user_id` and we populate it each time we build a new feature.

---

In [47]:
# empty user dataframe
users_dataset_base = pd.DataFrame(original_dataset.user_id.unique(), columns = ["user_id"])


'''
    The first feature we want to build is:
    **Favorite genre (i.e., the genre on which the user spent the most time)**
    We can exploit the `explode` method of Pandas to get from each entry a new entry for each genre in the list:
'''

# create exploded dataframe
exploded_dataset = original_dataset.explode("genres")[["user_id", "genres", "duration"]]

# add together all values with same user_id and same genre
exploded_dataset = exploded_dataset.groupby(["user_id", "genres"]).sum()

# get most watched genre
exploded_dataset = exploded_dataset.groupby("user_id").idxmax()
exploded_dataset.duration = exploded_dataset.duration.apply(lambda row: row[1])

# rename column
exploded_dataset = exploded_dataset.rename(columns={'duration': 'favorite_genre'})

# merge into user dataset
users_dataset_base = users_dataset_base.merge(exploded_dataset, left_on = "user_id", right_index = True)

'''
    The second feature to extract is:
    **Average click duration**
    This is very fast as we have just to calculate the mean of the `duration` columns of every user:
'''

# get mean
mean_dataset = original_dataset.groupby("user_id").duration.mean().reset_index()

# rename column
mean_dataset = mean_dataset.rename(columns={'duration': 'mean_duration'})

# merge into user dataset
users_dataset_base = users_dataset_base.merge(mean_dataset)

'''
    The third feature feature is:
    **Time of the day (Morning/Afternoon/Night) when the user spends the most time on the platform (the time spent is tracked through the duration of the clicks)**

    We will use as delimitations between the three time parts:
    - Morning: 6 to 14
    - Afternoon: 14 to 20
    - Night: 20 to 6
'''

# create working dataframe substituting the whole datetime with hour
hours = original_dataset[["user_id", "datetime", "duration"]].copy()
hours.datetime = hours.datetime.dt.hour

# cluster hours into three time periods
hours.datetime = hours.datetime.apply(lambda x: 'morning' if 6 <= x < 14 else 'afternoon' if 14 <= x < 20 else 'night')

# get most active time period for each user
hours = hours.groupby(["user_id", "datetime"]).sum().groupby("user_id").idxmax()
hours.duration = hours.duration.apply(lambda row: row[1])

# rename column
hours = hours.rename(columns={'duration': 'favorite_time_day'})

# merge into user dataset
users_dataset_base = users_dataset_base.merge(hours, left_on = "user_id", right_index = True)

'''
    The fourth feature is:
    **Is the user an old movie lover, or is he into more recent stuff (content released after 2010)?**
'''

# convert all datetimes to only the year 
years = original_dataset[["user_id", "release_date"]].copy()
years.release_date = years.release_date.dt.year

# check if film is old or not
years["new"] = years.release_date > 2010
years["old"] = years.release_date <= 2010

# count for each user how many have release year > 2010 and how many not, use boolean value as new column
user_oldnew = years.groupby("user_id").old.sum() > years.groupby("user_id").new.sum()

# rename column
user_oldnew.name = "is_oldmovies_lover"

# merge into user dataset
users_dataset_base = users_dataset_base.merge(user_oldnew, left_on = "user_id", right_index = True)

'''
    The fifth feature is:
    **Average time spent a day by the user (considering only the days he logs in)**
'''

# we can just take the mean_duration column and divide it by the number of different days the same user has in the dataset.

# we count how many different days every user has logged in
days = original_dataset[["user_id", "datetime"]].copy()
days["datetime"] = days["datetime"].dt.floor('D')
days = days.groupby("user_id").datetime.nunique()

# we count the total duration per user
sums = original_dataset.groupby("user_id").duration.sum()

# divide total duration by number of days
average_per_day = sums / days

# rename column
average_per_day.name = "duration_per_day"

# merge into user dataset
users_dataset_base = users_dataset_base.merge(average_per_day, left_on = "user_id", right_index = True)

In [48]:
users_dataset_base

,user_id,favorite_genre,mean_duration,favorite_time_day,is_oldmovies_lover,duration_per_day
0,1dea19f6fe,Drama,0.000000,night,True,0.000000
1,544dcbc510,Fantasy,1553.750000,afternoon,False,18645.000000
2,7cbcc791bf,Thriller,4218.666667,afternoon,False,8437.333333
3,ebf43c36b6,Action,1883.533333,afternoon,False,9417.666667
4,a57c992287,Drama,3684.538462,afternoon,False,7369.076923
...,...,...,...,...,...,...
161913,45414be0ec,Romance,0.000000,afternoon,False,0.000000
161914,783ec67e84,Drama,0.000000,afternoon,True,0.000000
161915,89c715f3a4,Adventure,-1.000000,afternoon,False,-1.000000
161916,9207e1499b,Crime,42592.000000,night,False,42592.000000


---

The additional features we propose are:

- `watchtime`: the total watchtime of a certain user. It's the sum of all the durations.
- `iterations`: the total number of iterations of a certain user. It's the number of rows related to that user.
- `active_days`: the number of days the user has been active on the site. 
- `n_movies`: the number of unique movies watched by the user
- `max_rewatches`: the maximum number of times the user has watched the same movie
- `favorite_day`: the user favourite day of the week to watch movies. We search it by finding the day with the highest duration overall for that user
- `favorite_month`: the user favourite month of the year to watch movies. We search it by finding the month with the highest duration overall for that user
- `most_active_year`: the year the user has been most active on based on the duration.
- `is_weekend_watcher`: if the user is a weekend watcher. We define a user "weekend watcher" if at least 50% of it's views are on saturday or sunday
- `longest_iteration`: the longest iteration registered for the user. We search it by finding the max across all durations

---

In [49]:
# create empty user dataframe
users_dataset_extended = pd.DataFrame(original_dataset.user_id.unique(), columns = ["user_id"])

# watchtime feature
n_movies = original_dataset.groupby("user_id").duration.sum()
n_movies.name = "total watchtime"

users_dataset_extended = users_dataset_extended.merge(n_movies, left_on = "user_id", right_index = True)

# number of iterations feature
n_iterations = original_dataset.groupby("user_id").size()
n_iterations.name = "iterations"

users_dataset_extended = users_dataset_extended.merge(n_iterations, left_on = "user_id", right_index = True)

# number of active days feature
days_active = original_dataset[["user_id", "datetime"]].copy()
days_active["datetime"] = days_active["datetime"].dt.floor('D')
days_active = days_active.groupby("user_id").datetime.nunique()
days_active.name = "active_days"

users_dataset_extended = users_dataset_extended.merge(days_active, left_on = "user_id", right_index = True)

# number of unique movies feature
n_movies = original_dataset.groupby("user_id").movie_id.nunique()
n_movies.name = "n_movies"
users_dataset_extended = users_dataset_extended.merge(n_movies, left_on = "user_id", right_index = True)

# highest number of rewatches feature
rewatch = original_dataset.groupby(["user_id", "movie_id"]).size().groupby("user_id").max()
rewatch.name = "max_rewatches"

users_dataset_extended = users_dataset_extended.merge(rewatch, left_on = "user_id", right_index = True)

# favorite day of the week feature
favourite_day = original_dataset[["user_id", "datetime"]].copy()
favourite_day.datetime = favourite_day.datetime.dt.dayofweek
favourite_day = favourite_day.groupby(["user_id", "datetime"]).size().groupby("user_id").idxmax().apply(lambda x: x[1])
favourite_day.name = "favorite_day"

users_dataset_extended = users_dataset_extended.merge(favourite_day, left_on = "user_id", right_index = True)

# favorite month of the year feature
favourite_month = original_dataset[["user_id", "datetime"]].copy()
favourite_month.datetime = favourite_month.datetime.dt.month
favourite_month = favourite_month.groupby(["user_id", "datetime"]).size().groupby("user_id").idxmax().apply(lambda x: x[1])
favourite_month.name = "favorite_month"

users_dataset_extended = users_dataset_extended.merge(favourite_month, left_on = "user_id", right_index = True)

# most active year feature
most_active_year = original_dataset[["user_id", "datetime"]].copy()
most_active_year.datetime = most_active_year.datetime.dt.year
most_active_year = most_active_year.groupby(["user_id", "datetime"]).size().groupby("user_id").idxmax().apply(lambda x: x[1])
most_active_year.name = "most_active_year"

users_dataset_extended = users_dataset_extended.merge(most_active_year, left_on = "user_id", right_index = True)

# weekend person feature
def weekend_function(row):
    row = row.droplevel(0)
    row = row.reindex(range(7), fill_value=0)
    return (row[5] + row[6]) > (row[0] + row[1] + row[2] + row[3] + row[4])

weekend = original_dataset[["user_id", "datetime"]].copy()
weekend.datetime = weekend.datetime.dt.dayofweek
weekend = weekend.groupby(["user_id", "datetime"]).size()
weekend = weekend.groupby("user_id").apply(weekend_function)
weekend.name = "is_weekend_user"

users_dataset_extended = users_dataset_extended.merge(weekend, left_on = "user_id", right_index = True)

# longest iteration feature
longest_iteration = original_dataset.groupby("user_id").duration.max()
longest_iteration.name = "longest_iteration"

users_dataset_extended = users_dataset_extended.merge(longest_iteration, left_on = "user_id", right_index = True)

In [50]:
users_dataset_extended

,user_id,total watchtime,iterations,active_days,n_movies,max_rewatches,favorite_day,favorite_month,most_active_year,is_weekend_user,longest_iteration
0,1dea19f6fe,0.0,1,1,1,1,6,1,2017,True,0.0
1,544dcbc510,18645.0,12,1,8,3,6,1,2017,True,10933.0
2,7cbcc791bf,25312.0,6,3,4,3,6,1,2017,False,10530.0
3,ebf43c36b6,28253.0,15,3,12,2,6,1,2017,True,7524.0
4,a57c992287,95798.0,26,13,22,3,0,1,2017,False,56338.0
...,...,...,...,...,...,...,...,...,...,...,...
161913,45414be0ec,0.0,1,1,1,1,6,6,2019,True,0.0
161914,783ec67e84,0.0,1,1,1,1,6,6,2019,True,0.0
161915,89c715f3a4,-1.0,1,1,1,1,6,6,2019,True,-1.0
161916,9207e1499b,42592.0,1,1,1,1,6,6,2019,True,42592.0


---

And finally we merge the two datasets.

---

In [60]:
users_dataset = users_dataset_base.merge(users_dataset_extended)
users_dataset

,user_id,favorite_genre,mean_duration,favorite_time_day,is_oldmovies_lover,duration_per_day,total watchtime,iterations,active_days,n_movies,max_rewatches,favorite_day,favorite_month,most_active_year,is_weekend_user,longest_iteration
0,1dea19f6fe,Drama,0.000000,night,True,0.000000,0.0,1,1,1,1,6,1,2017,True,0.0
1,544dcbc510,Fantasy,1553.750000,afternoon,False,18645.000000,18645.0,12,1,8,3,6,1,2017,True,10933.0
2,7cbcc791bf,Thriller,4218.666667,afternoon,False,8437.333333,25312.0,6,3,4,3,6,1,2017,False,10530.0
3,ebf43c36b6,Action,1883.533333,afternoon,False,9417.666667,28253.0,15,3,12,2,6,1,2017,True,7524.0
4,a57c992287,Drama,3684.538462,afternoon,False,7369.076923,95798.0,26,13,22,3,0,1,2017,False,56338.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161913,45414be0ec,Romance,0.000000,afternoon,False,0.000000,0.0,1,1,1,1,6,6,2019,True,0.0
161914,783ec67e84,Drama,0.000000,afternoon,True,0.000000,0.0,1,1,1,1,6,6,2019,True,0.0
161915,89c715f3a4,Adventure,-1.000000,afternoon,False,-1.000000,-1.0,1,1,1,1,6,6,2019,True,-1.0
161916,9207e1499b,Crime,42592.000000,night,False,42592.000000,42592.0,1,1,1,1,6,6,2019,True,42592.0


### 2.2 Choose your features (variables)!

You may notice that you have plenty of features to work with now. So, it would be best to find a way to reduce the dimensionality (reduce the number of variables to work with). You can follow the subsequent directions to achieve it:

1)  *To normalise or not to normalise? That's the question*. Sometimes, it is worth normalizing (scaling) the features. Explain if it is a good idea to perform any normalization method. If you think the normalization should be used, apply it to your data (look at the available normalization functions in the `scikit-learn` library).

2)  Select **one** method for dimensionality reduction and apply it to your data. Some suggestions are Principal Component Analysis, Multiple Correspondence Analysis, Singular Value Decomposition, Factor Analysis for Mixed Data, Two-Steps clustering. Make sure that the method you choose applies to the features you have or modify your data to be able to use it. Explain why you chose that method and the limitations it may have.

---

We will use as dimensionality reduction method the **Principal Component Analysis**. As the request make us choce only one dimensionality reduction method we will have to remove the two categorial features we have inside our dataset, `favorite_time_day` and `favorite_genre`.

---

In [61]:
users_dataset.drop(["favorite_genre", "favorite_time_day"], axis = 1, inplace = True)
users_dataset.set_index('user_id', inplace=True)
users_dataset

,mean_duration,is_oldmovies_lover,duration_per_day,total watchtime,iterations,active_days,n_movies,max_rewatches,favorite_day,favorite_month,most_active_year,is_weekend_user,longest_iteration
user_id,,,,,,,,,,,,,
1dea19f6fe,0.000000,True,0.000000,0.0,1,1,1,1,6,1,2017,True,0.0
544dcbc510,1553.750000,False,18645.000000,18645.0,12,1,8,3,6,1,2017,True,10933.0
7cbcc791bf,4218.666667,False,8437.333333,25312.0,6,3,4,3,6,1,2017,False,10530.0
ebf43c36b6,1883.533333,False,9417.666667,28253.0,15,3,12,2,6,1,2017,True,7524.0
a57c992287,3684.538462,False,7369.076923,95798.0,26,13,22,3,0,1,2017,False,56338.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
45414be0ec,0.000000,False,0.000000,0.0,1,1,1,1,6,6,2019,True,0.0
783ec67e84,0.000000,True,0.000000,0.0,1,1,1,1,6,6,2019,True,0.0
89c715f3a4,-1.000000,False,-1.000000,-1.0,1,1,1,1,6,6,2019,True,-1.0


---

We  beleive that normalization is needed in our dataset because it makes the weight a certain feature has on the distance between points indipendent from the values that feature treats. If for  xample we have a certain arbitrary feature orders of magnitude greater than any other without normalization that would become the only significant component when calculating distance.

We decided to use the standardization as normalizing function, to every feature we subtract the mean and divide by the standard deviation. This normalization fuction is available as `sklearn.preprocessing.StandardScaler`.

---

In [63]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_data = scaler.fit_transform(users_dataset)
scaled_users_dataset = pd.DataFrame(scaled_data, columns = users_dataset.columns)
scaled_users_dataset

,mean_duration,is_oldmovies_lover,duration_per_day,total watchtime,iterations,active_days,n_movies,max_rewatches,favorite_day,favorite_month,most_active_year,is_weekend_user,longest_iteration
0,-0.262460,2.007893,-0.288760,-0.373700,-0.480402,-0.539466,-0.491091,-0.515130,1.412119,-1.462076,-1.171474,1.516133,-0.352242
1,-0.253090,-0.498034,-0.193209,-0.323529,1.197928,-0.539466,1.143182,0.998340,1.412119,-1.462076,-1.171474,1.516133,-0.315727
2,-0.237018,-0.498034,-0.245521,-0.305589,0.282475,0.171208,0.209312,0.998340,1.412119,-1.462076,-1.171474,-0.659573,-0.317073
3,-0.251101,-0.498034,-0.240497,-0.297676,1.655654,0.171208,2.077053,0.241605,1.412119,-1.462076,-1.171474,1.516133,-0.327113
4,-0.240239,-0.498034,-0.250995,-0.115924,3.333984,3.724579,4.411729,0.998340,-1.453497,-1.462076,-1.171474,-0.659573,-0.164079
...,...,...,...,...,...,...,...,...,...,...,...,...,...
161913,-0.262460,-0.498034,-0.288760,-0.373700,-0.480402,-0.539466,-0.491091,-0.515130,1.412119,0.017497,1.489264,1.516133,-0.352242
161914,-0.262460,2.007893,-0.288760,-0.373700,-0.480402,-0.539466,-0.491091,-0.515130,1.412119,0.017497,1.489264,1.516133,-0.352242
161915,-0.262466,-0.498034,-0.288765,-0.373702,-0.480402,-0.539466,-0.491091,-0.515130,1.412119,0.017497,1.489264,1.516133,-0.352245
161916,-0.005593,-0.498034,-0.070487,-0.259092,-0.480402,-0.539466,-0.491091,-0.515130,1.412119,0.017497,1.489264,1.516133,-0.209989


---

And we execute the principal component analysis

---

In [71]:
from sklearn.decomposition import PCA
model = PCA(n_components = 7)
model.fit(scaled_users_dataset)
print(model.explained_variance_ratio_.sum())

0.931745542116829


In [72]:
pca_users_dataset = pd.DataFrame(model.transform(scaled_users_dataset))
pca_users_dataset

,0,1,2,3,4,5,6
0,-1.220113,-0.594497,1.949122,-0.116857,1.921583,-0.856963,1.783640
1,0.247025,1.259468,2.135746,0.322689,-0.322684,0.208478,1.994292
2,0.008452,0.908824,0.526405,0.295495,-0.259049,0.469726,1.997717
3,0.736877,1.939737,2.185442,0.332031,-0.498329,-0.946034,1.776856
4,3.603943,5.448190,-1.069554,0.350815,-0.427975,-1.780785,1.451653
...,...,...,...,...,...,...,...
161913,-1.231643,-0.558274,1.993906,1.116751,-0.455734,0.054265,-0.962098
161914,-1.280490,-0.621236,2.047033,0.714025,1.978300,-0.351485,-1.098012
161915,-1.231649,-0.558269,1.993906,1.116751,-0.455734,0.054265,-0.962098
161916,-0.946532,-0.792869,1.995997,1.114713,-0.454810,0.059737,-0.965735


---

limitations of PCA: only linear dependence

---

### 2.3 Clustering!

1)  Implement the K-means clustering algorithm (**not** ++: random initialization) using MapReduce. We ask you to write the algorithm from scratch following what you learned in class.

2)  Find an optimal number of clusters. Use at least two different methods. If your algorithms provide diverse optimal K's, select one of them and explain why you chose it.

3)  Run the algorithm on the data obtained from the dimensionality reduction.

4)  Implement **K-means++** from scratch and explain the differences with the results you got earlier.

5)  Ask ChatGPT to recommend other clustering algorithms and choose one. Explain your choice, then ask ChatGPT to implement it or use already implemented versions (e.g., the one provided in the scikit-learn library) and run it on your data. Explain the differences (if there are any) in the results. Which one is the best, in your opinion, and why?

In [ ]:
# TODO

### 2.4 Analysing your results!

You are often encouraged to explain the main characteristics that your clusters have. The latter is called the *Characterizing Clusters* step. Thus, follow the next steps to do it:

1)  Select 2-3 variables you think are relevant to identify the cluster of the customer. For example, Time_Day, Average Click Duration, etc.

2)  Most of your selected variables will be numerical (continuous or discrete), then categorize them into four categories.

3)  With the selected variables, perform pivot tables. On the horizontal axis, you will have the clusters, and on the vertical axis, you will have the categories of each variable. Notice that you have to do one pivot table per variable.

4)  Calculate the percentage by column for each pivot table. The sum of each row (cluster) must be 100.

5)  Interpret the results for each pivot table.

6)  Use any known metrics to estimate clustering algorithm performance (how good are the clusters you found?). Comment on the results obtained.

In [ ]:
# TODO

## 3. Bonus Question

We remind you that we consider and grade the bonuses only if you complete the entire assignment.

[Density-based clustering](https://wires.onlinelibrary.wiley.com/doi/epdf/10.1002/widm.30) identifies clusters as regions in the data space with high point density that are separated from other clusters by regions of low point density. The data points in the separating regions of low point density are typically considered noise or outliers. Typical algorithms that fall into this category are [OPTICS](https://dl.acm.org/doi/pdf/10.1145/304181.304187) and [DBSCAN](https://cdn.aaai.org/KDD/1996/KDD96-037.pdf).

1)  Ask ChatGPT (or any other LLM tool) to list three algorithms for Density-Based Clustering. Choose one and use it on the same dataset you used in 2.3. Analyze your results: how different are they from the centroid-based version?

__Note__: You can implement your algorithm from scratch or use the one implemented in the scikit-learn library; the choice is up to you!

In [ ]:
# TODO

## 4. Command Line Question

Here is another command line question to enjoy. We previously stated that using the command line tools is a skill that Data Scientists must master.

In this question, you should use any command line tool that you know to answer the following questions using the same dataset that you have been using so far:
  + What is the most-watched Netflix title?
  + Report the average time between subsequent clicks on Netflix.com
  + Provide the ID of the user that has spent the most time on Netflix
    
__Important note:__ You may work on this question in any environment (AWS, your PC command line, Jupyter notebook, etc.), but the final script must be placed in CommandLine.sh, which must be executable. Please run the script and include a __screenshot__ of the <ins>output</ins> in the notebook for evaluation.  

In [ ]:
# TODO

## 5. Algorithmic Question 

Federico studies in a demanding university where he has to take a certain number $N$ of exams to graduate,  but he is free to choose in which order he will take these exams. Federico is panicking since this university is not only one of the toughest in the world but also one of the weirdest. His final grade won't depend at all on the mark he gets in these courses: there's a precise evaluation system. 

He was given an initial personal score of $S$ when he enrolled, which changes every time he takes an exam: now comes the crazy part.
He soon discovered that every of the $N$ exams he has to take is assigned a mark $p$. Once he has chosen an exam, his score becomes equal to the mark $p$, and at the same time, the scoring system changes:
+ If he takes an "easy" exam (the score of the exam being less than his score), every other exam's mark is increased by the quantity $S - p$.
+ If he takes a "hard" exam (the score of the exam is greater than his score), every other exam's mark is decreased by the quantity $p - S$.
  
So, for example, consider $S=8$ as the initial personal score. Federico must decide which exam he wants to take, being $[5,7,1]$ the marks list. If he takes the first one, being $5 < 8$ and $8 - 5 = 3$, the remaining list now becomes $[10,4]$, and his score is updated as $S = 5$.

In this chaotic university where the only real exam seems to be choosing the best way to take exams, you are the poor student advisor who is facing a long queue of confused people who need some help. Federico is next in line,  and he comes up in turn with an inescapable question: he wants to know which is the highest score possible he could get. 

a) Fortunately, you have a computer app designed by a brilliant student. Federico wants you to show him the code which this app is based on
because he wants to do paid counseling for other desperate students: in a *recursive* fashion, the helped helps the helpable.

b) Federico is getting angry because he claims that your code is slow! Show him formally with a big-O notation that he is as crazy as this university! 

c) If, unfortunately, Federico is right in the grip of madness, he will threaten you to optimize the code through a different approach. 
You should end this theater of the absurd by any means! (And again, formally prove that you improved time complexity)

d) Ask chatGPT for a third (optimized) implementation and analyze again its time complexity. Be careful (and crafty) in defining the prompt, and challenge the machine in this coding question!

Here are some input/output examples (the first value is the initial personal score, and the second line contains the list of marks): 

__Input 1__
```
8
5 7 1 
```

__Output 1__
```
11
```

__Input 2__
```
25
18 24 21 32 27
```

__Output 2__
```
44
```

__Input 3__
```
30
13 27 41 59 28 33 39 19 52 48 55 79
```

__Output 3__
```
205
```

**a) Fortunately, you have a computer app designed by a brilliant student. Federico wants you to show him the code which this app is based on
because he wants to do paid counseling for other desperate students: in a *recursive* fashion, the helped helps the helpable.**

In [33]:
import numpy as np

def iterative_call(student_score, marks_list):
    if len(marks_list) == 1:
        return marks_list[0]
    else:
        best_score = 0
        for i in range(len(marks_list)):
            score = marks_list[i]
            new_marks_list = np.concatenate([marks_list[:i], marks_list[i+1:]])
            new_marks_list += (student_score - score)
            new_score = iterative_call(score, new_marks_list)

            best_score = max(new_score, best_score)
        
        return best_score

def algorithmic_question(input_string):
    # get first line of input
    input_list = input_string.split("\n")
    original_student_score = int(input_list[0])
    original_marks_list = np.array(list(map(int, input_list[1].split(" "))))

    best_score = iterative_call(original_student_score, original_marks_list)

    print(best_score)

input1 = "8\n5 7 1"
print("Test1:")
algorithmic_question(input1)

input2 = "25\n18 24 21 32 27"
print("Test2:")
algorithmic_question(input2)

input3 = "30\n13 27 41 59 28 33 39 19 52 48 55 79"
print("Test3:")
algorithmic_question(input3)

Test1:
11
Test2:
44
Test3:
205


**b) Federico is getting angry because he claims that your code is slow! Show him formally with a big-O notation that he is as crazy as this university!**

---

**Time complexity:**

We will analyze the computational time of the algorithm in Big-O notation in the `worst case scenario`.

This algorithm explores all possible permutations of marks choices using recurive calls to avoid having to calculate the same parts parts more than once. Therefore to calculate the time complexity of the algorithm we have to find the cost of a `iterative_call` and how many calls the algorithm is gonna do. The number of iterative calls will be not only the number of permutations of `n` numbers, but we have also to consider "intermediate" operations. We have `n` ways to set the first mark, for each of those we have `n - 1` ways to set the second and so on...

$n * (n - 1) * (n - 2) * ... * 2 * 1 = n!$

To study the `iterative_call` function we will say that the current marks list has length `i`. In the worst case scenario this function is composed of a condition check on the length of the array (constant time) and a cycle trough the current elements of marks list (`i` repetitions) that for every cycle create a new array (linear cost `O(i)`), an update operation on the new array (linear cost `O(i)`), the next iterative call (that will have length `i - 1`) and a max operation (constant time). Ignoring the recursive call we can say that the function has cost `O(i)` with `i` length of the input.

We can see that the real value of the iterative call is not important, as for the extreme case where `i = constant` the total cost of the algorithm will be:

$n! * constant \in O(n!)$

And in the opposite extreme where `i = n` the total cost will be:

$n! * n < n! * (n + 1) = (n + 1)! \in O((n + 1)!) = O(n!)$

And that is an unfeasable computational cost.

---

**c) If, unfortunately, Federico is right in the grip of madness, he will threaten you to optimize the code through a different approach. 
You should end this theater of the absurd by any means! (And again, formally prove that you improved time complexity)**

---

This new algorithm is based on the following observations on every permutation of the courses that yields the best output:
* They are an alternation of decreases and increases
* They start with a decrease and end with an increase
* The only time there are two actions in the same side is when the number of marks is uneven

Therefore to find the highest output value we can find the median, divide the marks list in two sets (one $\le$ the median and one $>$) and  select them in alternating order.


---

In [24]:
import numpy as np

def algorithmic_question(input_string):
    # get first line of input
    input_list = input_string.split("\n")
    student_score = int(input_list[0])

    # get marks list
    marks_list = np.array(list(map(int, input_list[1].split(" "))))
    marks_list.sort()

    # if treat case uneven marks list
    if len(marks_list) % 2 == 1:
        # we will call those three lines "update operation"
        new_score = marks_list[len(marks_list) // 2]
        marks_list += (student_score - new_score)
        student_score = new_score

    # select them in alternating order
    for i in range(len(marks_list) // 2):
        new_score = marks_list[i]
        marks_list += (student_score - new_score)
        student_score = new_score

        new_score = marks_list[-i -1]
        marks_list += (student_score - new_score)
        student_score = new_score
    
    print(student_score)

input1 = "8\n5 7 1"
print("Test1:")
algorithmic_question(input1)

input2 = "25\n18 24 21 32 27"
print("Test2:")
algorithmic_question(input2)

input3 = "30\n13 27 41 59 28 33 39 19 52 48 55 79"
print("Test3:")
algorithmic_question(input3);

Test1:
11
Test2:
44
Test3:
205


---

**Time complexity:**

We will analyze the computational time of the algorithm in Big-O notation in the `worst case scenario`.

We can see the algorithm as divided in three principal components with their cost proportional to the size number of marks in the input, that we will call `n`.

* The first thing the algorithm do is read all the input, for which it takes time proportional to the size of the input, `O(n)`
* After reading all the input the marks list is sorted to find the median, this will take `O(nlogn)` time
* Finally for every mark the "update operation" is executed exactly one time. This update operation consist in reading the new mark score (costant time operation considering numpy arrays), updating every element in the marks list (which cost is linear) and finally updating the student score (constant time). Therefore the cost of each "update operation" is to be considered linear, `O(n)`. We can observe that deleting an element from the marks list after it is used will not help us reduce the computational cost, as also the delete operation as a linear cost or even updating only the unseen values, as the cost of `n` executions will always remain `O(n^2)`.

The cost of the algorithm is therefore `O(n^2)` as the update operation will always be executed `n` times.

We can observe that a study in the `average case scenario` would lead to a change of the computational cost in Big-O notation. As we are not interested in ordering the array but only find the median we could use a `quickselect` algorithm to find the median (average cost `O(n)`, worst case costs `O(n^2)`).

---

**d) Ask chatGPT for a third (optimized) implementation and analyze again its time complexity. Be careful (and crafty) in defining the prompt, and challenge the machine in this coding question!**

---

We asked Chat GPT 3.5 to write an algorithm using the following prompt:

```
Resolve the following problem with a python algorithm giving particular attention to code correctness and computational cost:
[The proposed algorithm text, including the examples]
```

And the received output is:

```
This problem can be approached by iterating through the exams and determining whether an exam is "easy" or "hard" based on Federico's score and the exam's mark. Then, update the scores of the remaining exams accordingly. Here's a Python algorithm to solve this problem efficiently:

def calculate_final_score(S, marks):
    final_score = 0
    for mark in marks:
        if mark < S:
            final_score += S - mark
        else:
            final_score += mark - S
        # Update S for the next iteration
        S = mark
    return final_score

# Test cases
inputs = [
    (8, [5, 7, 1]),
    (25, [18, 24, 21, 32, 27]),
    (30, [13, 27, 41, 59, 28, 33, 39, 19, 52, 48, 55, 79])
]

for S, marks in inputs:
    result = calculate_final_score(S, marks)
    print(result)
```

Which is both wrong and tries to mimick the recursive resolution of the problem. We therefore tried to rewrite the problem text in a more understandable way:

```
You are asked to write an optimized algorithm in python to solve the following problem: 

You start with a score "S" and a list of "N" other marks.  Everytime you take a score from the list (we will call it "p") you have to sum to every other exam remaining in the list the difference between S and p and then update S with the mark you extracted (S = p).

You have to take all the elements from the list one by one in such a way that maximizes the value "S" has at the end of the execution.

Here are some examples:

[Same examples as before]
```

And the obtained response:

```
Sure, here's a Python solution to solve this problem using a greedy algorithm approach:

def maximize_score(S, marks):
    marks.sort(reverse=True)  # Sort marks in descending order
    total = 0

    for mark in marks:
        total += S * len(marks)  # Update total by adding S times the number of remaining marks
        S = mark  # Update S with the current mark

    return total + S  # Add the final value of S to the total

# Example inputs
inputs = [
    (8, [5, 7, 1]),
    (25, [18, 24, 21, 32, 27]),
    (30, [13, 27, 41, 59, 28, 33, 39, 19, 52, 48, 55, 79])
]

# Calculate and print outputs
for i, (S, marks) in enumerate(inputs, 1):
    result = maximize_score(S, marks)
    print(f"Output {i}: {result}")


Sure, here's a Python solution to solve this problem using a greedy algorithm approach:

python
Copy code
def maximize_score(S, marks):
    marks.sort(reverse=True)  # Sort marks in descending order
    total = 0

    for mark in marks:
        total += S * len(marks)  # Update total by adding S times the number of remaining marks
        S = mark  # Update S with the current mark

    return total + S  # Add the final value of S to the total

# Example inputs
inputs = [
    (8, [5, 7, 1]),
    (25, [18, 24, 21, 32, 27]),
    (30, [13, 27, 41, 59, 28, 33, 39, 19, 52, 48, 55, 79])
]

# Calculate and print outputs
for i, (S, marks) in enumerate(inputs, 1):
    result = maximize_score(S, marks)
    print(f"Output {i}: {result}")
This code defines a maximize_score function that takes the initial score S and a list of marks as inputs. It sorts the marks in descending order and then iterates through each mark, updating the total score based on the described algorithm.

The approach here is to sort the marks in descending order and for each mark, add the current score S multiplied by the number of remaining marks to the total. Finally, add the last value of S to the total and return it as the result.

You can run this code with different inputs to get the corresponding outputs as described in your examples.
```

Which continues to be wrong and iterative. Therefore we tried to suggest him the reasoning behind the optimized solution:

```
You are asked to write an optimized algorithm in python to solve the following problem: 

You start with a score "S" and a list of "N" other marks.  Everytime you take a score from the list (we will call it "p") you have to update every exam remaining in the list by summing the difference between S and p to each value and then update S with the mark you extracted (S = p).

You have to take all the elements from the list one by one in such a way that maximizes the value "S" has at the end of the execution.

We have observed the following properties to all the permutations that yield the best possible output:
* They are an alternation of values below and above the median
* They start with a below and end with a value above
* The only time there are two successive elements are from the same side is when the number of marks is uneven

Here are some examples:

[Same examples as before]
```

Which received as output:

```
This problem seems like a variation of a greedy algorithm. To approach this in Python, we can follow these steps:

Sort the list of marks.
Find the median.
Create two lists: one for values below the median and one for values above it.
Generate the alternating sequence starting with a value below the median and ending with a value above it.
Update the score "S" and compute the maximum value.
Here's a Python implementation:

def maximize_score(S, marks):
    marks.sort()  # Sort the list of marks
    n = len(marks)
    median = marks[n // 2] if n % 2 != 0 else (marks[n // 2 - 1] + marks[n // 2]) // 2

    below_median = [mark for mark in marks if mark < median]
    above_median = [mark for mark in marks if mark > median]

    # Generating the alternating sequence
    sequence = []
    if n % 2 != 0:
        while below_median and above_median:
            sequence.append(below_median.pop())
            sequence.append(above_median.pop())
    else:
        while below_median and above_median:
            sequence.append(above_median.pop())
            sequence.append(below_median.pop())

    # Adding remaining elements if any
    sequence.extend(below_median[::-1])
    sequence.extend(above_median[::-1])

    # Calculating the maximum value of S
    for mark in sequence:
        S += mark - S

    return S

# Test cases
print(maximize_score(8, [5, 7, 1]))  # Output: 11
print(maximize_score(25, [18, 24, 21, 32, 27]))  # Output: 44
print(maximize_score(30, [13, 27, 41, 59, 28, 33, 39, 19, 52, 48, 55, 79]))  # Output: 205

This Python code should solve the problem using the defined approach and provide the maximum value of "S" based on the given list of marks and initial score.
```

That is still wrong, but the approach is more similar to the optimized one.

---